In [75]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import tensorflow

In [76]:
mt=pd.read_csv('data/meteorshowers.csv')
cities=pd.read_csv('data/cities.csv')
constellations=pd.read_csv('data/constellations.csv')
moon_phases=pd.read_csv('data/moonphases.csv')

In [77]:
mt

,name,radiant,bestmonth,startmonth,startday,endmonth,endday,hemisphere,preferredhemisphere
0,Lyrids,Lyra,april,april,21,april,22,northern,northern
1,Eta Aquarids,Aquarius,may,april,19,may,28,"northern, southern",southern
2,Orionids,Orion,october,october,2,november,7,"northern, southern","northern, southern"
3,Perseids,Perseus,august,july,14,august,24,northern,northern
4,Leonids,Leo,november,november,6,november,30,"northern, southern","northern, southern"


In [78]:
months={'january':1, 'february':2, 'march':3, 'april':4, 'may':5, 'june':6, 'july':7, 'august':8, 'september':9, 'october':10, 'november':11, 'december':12}

In [79]:
mt.bestmonth=mt.bestmonth.map(months)
mt.startmonth=mt.startmonth.map(months)

In [80]:
mt.endmonth=mt.endmonth.map(months)
moon_phases.month=moon_phases.month.map(months)
constellations.bestmonth=constellations.bestmonth.map(months)

In [81]:
mt['startdate']=pd.to_datetime(2020*10000+mt.startmonth*100+mt.startday, format='%Y%m%d')
mt['enddate']=pd.to_datetime(2020*10000+mt.endmonth*100+mt.endday, format='%Y%m%d')

In [82]:
moon_phases['date']=pd.to_datetime(2020*10000+moon_phases.month*100+moon_phases.month, format='%Y%m%d')

In [83]:
moon_phases

,month,day,moonphase,specialevent,date
0,1,1,NaN,NaN,2020-01-01
1,1,2,first quarter,NaN,2020-01-01
2,1,3,NaN,NaN,2020-01-01
3,1,4,NaN,NaN,2020-01-01
4,1,5,NaN,NaN,2020-01-01
...,...,...,...,...,...
361,12,27,NaN,NaN,2020-12-12
362,12,28,NaN,NaN,2020-12-12
363,12,29,full moon,NaN,2020-12-12
364,12,30,NaN,NaN,2020-12-12


In [84]:
hemispheres={'northern':0, 'southern':1, 'northern, southern':3}

In [85]:
mt.hemisphere=mt.hemisphere.map(hemispheres)
constellations.hemisphere=constellations.hemisphere.map(hemispheres)

In [86]:
mt

,name,radiant,bestmonth,startmonth,startday,endmonth,endday,hemisphere,preferredhemisphere,startdate,enddate
0,Lyrids,Lyra,4,4,21,4,22,0,northern,2020-04-21,2020-04-22
1,Eta Aquarids,Aquarius,5,4,19,5,28,3,southern,2020-04-19,2020-05-28
2,Orionids,Orion,10,10,2,11,7,3,"northern, southern",2020-10-02,2020-11-07
3,Perseids,Perseus,8,7,14,8,24,0,northern,2020-07-14,2020-08-24
4,Leonids,Leo,11,11,6,11,30,3,"northern, southern",2020-11-06,2020-11-30


In [87]:
phases={'new moon':0, 'first quarter':0.5, 'third quarter':0.5, 'full moon':1.0}

In [88]:
moon_phases['percentage']=moon_phases.moonphase.map(phases)

In [89]:
moon_phases = moon_phases.drop(['day','month','moonphase','specialevent'],axis=1)

In [90]:
lastPhase = 0

for index, row in moon_phases.iterrows():
    if pd.isnull(row['percentage']):
        moon_phases.at[index, 'percentage'] = lastPhase
    else:
        lastPhase = row['percentage']

In [91]:
moon_phases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        366 non-null    datetime64[ns]
 1   percentage  366 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.8 KB


In [92]:
mt = mt.drop(['startmonth','startday','endmonth','endday','hemisphere'],axis=1)

In [93]:
moon_phases

,date,percentage
0,2020-01-01,0.0
1,2020-01-01,0.5
2,2020-01-01,0.5
3,2020-01-01,0.5
4,2020-01-01,0.5
...,...,...
361,2020-12-12,0.5
362,2020-12-12,0.5
363,2020-12-12,1.0
364,2020-12-12,1.0


In [107]:
def predict_best_meteor_shower_viewing(city):
    meteor_shower_string = ""

    if city not in cities.values:
        meteor_shower_string = "Unfotunately, "+ city + "isn't available for prediction now!"
        return meteor_shower_string

    latitude = cities.loc[cities['city'] == city, 'latitude'].iloc[0]
    constellation_list = constellations.loc[(constellations['latitudestart']>=latitude) & (constellations['latitudeend']<=latitude), 'constellation'].to_list()

    if not constellation_list:
        meteor_shower_string = "Unfotunately, there are no meteor shower viewable from "+ city + "."
        return meteor_shower_string

    for constellation in constellation_list:
        meteor_shower = mt.loc[mt['radiant'] == constellation, 'name'].iloc[0]
        meteor_shower_startdate = mt.loc[mt['radiant'] == constellation, 'startdate'].iloc[0]
        meteor_shower_enddate = mt.loc[mt['radiant'] == constellation, 'enddate'].iloc[0]
        moon_phases_list = moon_phases.loc[(moon_phases['date'] >= meteor_shower_startdate) & (moon_phases['date'] <= meteor_shower_enddate)]
        
        if moon_phases_list: 
            best_moon_date = moon_phases_list.loc[moon_phases_list['percentage'].idxmin()]['date']

        meteor_shower_string += meteor_shower + " is best seen if you look towards the " + constellation + " constellation on " + best_moon_date.to_pydatetime().strftime("%B %d, %Y") + "./n"

        return meteor_shower_string

    

In [109]:
print(predict_best_meteor_shower_viewing("Rabat"))

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().